In [ ]:
from cryptography.fernet import Fernet  # Importa a biblioteca Fernet para criptografia simétrica
import pandas as pd  
import ipywidgets as widgets  # Importa a biblioteca ipywidgets para criar widgets interativos
from IPython.display import display  # Importa a função display para exibir os widgets e outros elementos no Jupyter Notebook
import os  

# Funções

# Função para gerar a chave de criptografia
def gerar_chave():
    key = Fernet.generate_key()  # Gera uma chave aleatória usando o Fernet
    with open('chave.key', 'wb') as key_file:  # Cria o arquivo 'chave.key' no modo de escrita binária (wb)
        key_file.write(key)  # Escreve a chave gerada no arquivo
    print('Chave gerada e salva como chave.key ✅') 

# Função para carregar a chave de criptografia
def carregar_chave(caminho_chave=None):
    if os.path.exists('chave.key'):  # Se o arquivo 'chave.key' existir no diretório, abre o arquivo 
        with open('chave.key', 'rb') as file:  
            return Fernet(file.read())  
    else:  
        print("Arquivo de chave não encontrado ❌") 
        return None

# Função para criptografar 
def criptografar_base(caminho_csv):
    df = pd.read_csv(caminho_csv)  # Carrega o arquivo CSV em um DataFrame
    print("Base original:") 
    display(df)  # Exibe a base original no notebook

    fernet = carregar_chave()  # Carrega a chave de criptografia
    if fernet is None:  # Se a chave não foi carregada corretamente
        return  # Sai da função sem fazer mais nada

    # Itera sobre as colunas da base de dados (ignorando a primeira coluna)
    for coluna in df.columns[1:]:
        # Aplica a criptografia em cada valor da coluna
        df[coluna] = df[coluna].apply(lambda x: fernet.encrypt(str(x).encode()).decode())

    df.to_csv('dados_criptografados.csv', index=False) 
    print("🔐 Base criptografada salva como dados_criptografados.csv")  

# Função para descriptografar 
def descriptografar_base(caminho_csv, caminho_chave=None):
    df = pd.read_csv(caminho_csv) 
    print("Base criptografada:")  
    display(df) 

    fernet = carregar_chave(caminho_chave)  # Carrega a chave de criptografia
    if fernet is None:  # Se a chave não foi carregada corretamente
        return  # Sai da função sem fazer mais nada

    # Itera sobre as colunas da base de dados (ignorando a primeira coluna)
    for coluna in df.columns[1:]:
        # Aplica a descriptografia em cada valor da coluna
        df[coluna] = df[coluna].apply(lambda x: fernet.decrypt(str(x).encode()).decode())

    df.to_csv('dados_descriptografados.csv', index=False)  
    print("🔓 Base descriptografada salva como dados_descriptografados.csv")


# Interface

# Cria um menu para selecionar a ação desejada: gerar chave, criptografar ou descriptografar
acao_dropdown = widgets.Dropdown(
    options=[
        ('Selecionar...', '0'),
        ('Gerar chave de criptografia', '1'),
        ('Criptografar uma base de dados', '2'),
        ('Descriptografar uma base de dados', '3')
    ],
    description='Ação:', 
    style={'description_width': 'initial'}
)

# Cria um campo de upload de arquivo que será criptografado ou descriptografado
caminho_arquivo = widgets.FileUpload(
    description='Escolha o CSV:',  
    style={'description_width': 'initial'},  
)

# Cria um campo de upload de arquivo para o usuário escolher a chave
caminho_chave = widgets.FileUpload(
    description='Escolha a chave (chave.key):',  
    style={'description_width': 'initial'}, 
)

# Cria um botão para o usuário executar a ação selecionada
botao_executar = widgets.Button(
    description="Executar",  
    button_style='success'  
)

# Cria uma área de saída onde as mensagens serão exibidas
saida = widgets.Output()

# Função que é chamada quando o botão "Executar" é clicado
def ao_clicar(botao):
    with saida:
        saida.clear_output()  # Limpa a saída anterior
        opcao = acao_dropdown.value  # Obtém o valor do menu
        caminho = list(caminho_arquivo.value.keys())[0] if caminho_arquivo.value else ''  # Obtém o caminho do arquivo
        chave = list(caminho_chave.value.keys())[0] if caminho_chave.value else None  # Obtém a chave

        # Verifica qual ação foi selecionada e executa a função correspondente
        if opcao == '1':  # Gerar chave
            gerar_chave()
        elif opcao == '2':  # Criptografar base
            if not os.path.exists(caminho):  # Verifica se o arquivo existe
                print("❌ Arquivo não encontrado.")  # Se não encontrado, exibe um erro
                return
            criptografar_base(caminho)  # Chama a função de criptografar
        elif opcao == '3':  # Descriptografar base
            if not os.path.exists(caminho):  # Verifica se o arquivo existe
                print("❌ Arquivo não encontrado.")  # Se não encontrado, exibe um erro
                return
            if chave:  # Verifica se a chave foi fornecida
                # Cria um arquivo temporário para armazenar a chave fornecida
                caminho_chave_temp = 'temp_chave.key'
                with open(caminho_chave_temp, 'wb') as f:
                    f.write(caminho_chave.value[chave]['content'])  # Escreve a chave fornecida no arquivo temporário
                descriptografar_base(caminho, caminho_chave_temp)  # Chama a função de descriptografar
                os.remove(caminho_chave_temp)  # Apaga o arquivo temporário após a descriptografia
            else:
                print("❌ Chave não fornecida para descriptografar.")  # Se a chave não for fornecida, exibe um erro
        else:  # Caso nenhuma ação válida seja selecionada
            print("⚠️ Por favor, selecione uma ação válida.")  # Exibe um aviso informando para selecionar uma ação válida

# Vincula a função 'ao_clicar' ao evento de clique no botão
botao_executar.on_click(ao_clicar)

# Exibe os widgets no notebook para o usuário interagir
display(widgets.VBox([acao_dropdown, caminho_arquivo, caminho_chave, botao_executar, saida]))
